### Part 2 Web scrapping of Population and Demographics data of New York city from Wikipedia

In [ ]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#### A : POPULATION DATA

Web scrapping of Population data from wikipedia page - https://en.wikipedia.org/wiki/New_York_City

#### Download all the dependencies that is needed.

In [6]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    ------------------------------------------------------------
                       

In [8]:
website_url = requests.get('https://en.wikipedia.org/wiki/Demographics_of_New_York_City').text
soup = BeautifulSoup(website_url,'lxml')
import csv
table = soup.find('table',{'class':'wikitable sortable'})
#print(soup.prettify())

headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

with open('POPULATION1.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)

In [28]:
Pop_data=pd.read_csv('POPULATION1.csv')
Pop_data.drop(Pop_data.columns[[3,4,7,8,9,10,11,12,13,14]], axis=1,inplace=True)
Pop_data
#print('Data downloaded!')

,New York City's five boroughsvte,Jurisdiction,Population,Density,Borough
0,The Bronx\n,\n Bronx\n,"1,418,207\n",42.10\n,109.04\n
1,Brooklyn\n,\n Kings\n,"2,559,903\n",70.82\n,183.42\n
2,Manhattan\n,\n New York\n,"1,628,706\n",22.83\n,59.13\n
3,Queens\n,\n Queens\n,"2,253,858\n",108.53\n,281.09\n
4,Staten Island\n,\n Richmond\n,"476,143\n",58.37\n,151.18\n
5,City of New York,"8,336,817",842.343,783.83,"27,547"
6,State of New York,"19,453,561","1,731.910","122,284",412
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN


#### Remove whitespaces and rename columns

In [29]:
Pop_data.columns = Pop_data.columns.str.replace(' ', '')
Pop_data.columns = Pop_data.columns.str.replace('\'','')
Pop_data.rename(columns={'Borough':'persons_sq_mi'}, inplace=True)
Pop_data

,NewYorkCitysfiveboroughsvte,Jurisdiction,Population,Density,persons_sq_mi
0,The Bronx\n,\n Bronx\n,"1,418,207\n",42.10\n,109.04\n
1,Brooklyn\n,\n Kings\n,"2,559,903\n",70.82\n,183.42\n
2,Manhattan\n,\n New York\n,"1,628,706\n",22.83\n,59.13\n
3,Queens\n,\n Queens\n,"2,253,858\n",108.53\n,281.09\n
4,Staten Island\n,\n Richmond\n,"476,143\n",58.37\n,151.18\n
5,City of New York,"8,336,817",842.343,783.83,"27,547"
6,State of New York,"19,453,561","1,731.910","122,284",412
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN


In [30]:
Pop_data.rename(columns = {'NewYorkCitysfiveboroughsvte\n' : 'Borough',
                   'Jurisdiction\n':'County',
                   'Population\n':'Estimate_2019', 
                   'Density\n':'square_mi'}, inplace=True)
Pop_data

,Borough,County,Estimate_2019,square_mi,persons_sq_mi
0,The Bronx\n,\n Bronx\n,"1,418,207\n",42.10\n,109.04\n
1,Brooklyn\n,\n Kings\n,"2,559,903\n",70.82\n,183.42\n
2,Manhattan\n,\n New York\n,"1,628,706\n",22.83\n,59.13\n
3,Queens\n,\n Queens\n,"2,253,858\n",108.53\n,281.09\n
4,Staten Island\n,\n Richmond\n,"476,143\n",58.37\n,151.18\n
5,City of New York,"8,336,817",842.343,783.83,"27,547"
6,State of New York,"19,453,561","1,731.910","122,284",412
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN


#### Replace newline('\n') from each string from left and right sides

In [31]:
Pop_data['Borough']=Pop_data['Borough'].replace(to_replace='\n', value='', regex=True)
Pop_data['County']=Pop_data['County'].replace(to_replace='\n', value='', regex=True)
Pop_data['Estimate_2019']=Pop_data['Estimate_2019'].replace(to_replace='\n', value='', regex=True)
Pop_data['square_mi']=Pop_data['square_mi'].replace(to_replace='\n', value='', regex=True)
Pop_data['persons_sq_mi']=Pop_data['persons_sq_mi'].replace(to_replace='\n', value='', regex=True)
Pop_data

,Borough,County,Estimate_2019,square_mi,persons_sq_mi
0,The Bronx,Bronx,"1,418,207",42.10,109.04
1,Brooklyn,Kings,"2,559,903",70.82,183.42
2,Manhattan,New York,"1,628,706",22.83,59.13
3,Queens,Queens,"2,253,858",108.53,281.09
4,Staten Island,Richmond,"476,143",58.37,151.18
5,City of New York,"8,336,817",842.343,783.83,"27,547"
6,State of New York,"19,453,561","1,731.910","122,284",412
7,Sources:[14] and see individual borough articles,NaN,NaN,NaN,NaN


#### Shift data in the last two rows

In [33]:
#Pop_data.loc[5:,['persons_sq_mi','persons_sq_km']] = Pop_data.loc[2:,['persons_sq_mi','persons_sq_km']].shift(1,axis=1)
#Pop_data.loc[5:,['square_mi','persons_sq_mi']] = Pop_data.loc[2:,['square_km','persons_sq_mi']].shift(1,axis=1)
Pop_data.loc[5:,['square_mi','persons_sq_mi']] = Pop_data.loc[2:,['square_mi','persons_sq_mi']].shift(1,axis=1)
Pop_data.loc[5:,['Estimate_2019','square_mi']] = Pop_data.loc[2:,['Estimate_2019','square_mi']].shift(1,axis=1)
Pop_data.loc[5:,['County','Estimate_2019']] = Pop_data.loc[2:,['County','Estimate_2019']].shift(1,axis=1)
Pop_data.loc[5:,['Borough','County']] = Pop_data.loc[2:,['Borough','County']].shift(1,axis=1)
Pop_data

,Borough,County,Estimate_2019,square_mi,persons_sq_mi
0,The Bronx,Bronx,"1,418,207",42.10,109.04
1,Brooklyn,Kings,"2,559,903",70.82,183.42
2,Manhattan,New York,"1,628,706",22.83,59.13
3,Queens,Queens,"2,253,858",108.53,281.09
4,Staten Island,Richmond,"476,143",58.37,151.18
5,NaN,City of New York,"8,336,817",842.343,783.83
6,NaN,State of New York,"19,453,561","1,731.910","122,284"
7,NaN,Sources:[14] and see individual borough articles,NaN,NaN,NaN


#### Remove 'NAN'

In [34]:
Pop_data = Pop_data.fillna('')
Pop_data

,Borough,County,Estimate_2019,square_mi,persons_sq_mi
0,The Bronx,Bronx,"1,418,207",42.10,109.04
1,Brooklyn,Kings,"2,559,903",70.82,183.42
2,Manhattan,New York,"1,628,706",22.83,59.13
3,Queens,Queens,"2,253,858",108.53,281.09
4,Staten Island,Richmond,"476,143",58.37,151.18
5,,City of New York,"8,336,817",842.343,783.83
6,,State of New York,"19,453,561","1,731.910","122,284"
7,,Sources:[14] and see individual borough articles,,,


#### Drop the last row

In [36]:
i = Pop_data[((Pop_data.County == 'Sources:[14] and see individual borough articles'))].index
Pop_data.drop(i)

,Borough,County,Estimate_2019,square_mi,persons_sq_mi
0,The Bronx,Bronx,"1,418,207",42.10,109.04
1,Brooklyn,Kings,"2,559,903",70.82,183.42
2,Manhattan,New York,"1,628,706",22.83,59.13
3,Queens,Queens,"2,253,858",108.53,281.09
4,Staten Island,Richmond,"476,143",58.37,151.18
5,,City of New York,"8,336,817",842.343,783.83
6,,State of New York,"19,453,561","1,731.910","122,284"


#### Save dataframe as csv file

In [37]:
Pop_data.to_csv('POPULATION.csv',index=False)

#### B : DEMOGRAPHICS DATA

We will web scrap Demographics data from wikipedia page - https://en.wikipedia.org/wiki/New_York_City

#### Web scrapping of Demographics data from wikipedia page using BeautifulSoup.

Beautiful Soup is a Python package for parsing HTML and XML documents (including having malformed markup, i.e. non-closed tags, so named after tag soup). It creates a parse tree for parsed pages that can be used to extract data from HTML, which is useful for web scraping.

In [43]:
import folium # plotting library
from bs4 import BeautifulSoup

import csv
website_url = requests.get('https://en.wikipedia.org/wiki/New_York_City').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable sortable collapsible'})
#print(soup.prettify())

headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

with open('NYC_DEMO.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)

AttributeError: 'NoneType' object has no attribute 'find_all'